In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Обзор данных

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data/ml-0-users_behavior.csv')

In [ ]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
df.max()

calls         244.00
minutes      1632.06
messages      224.00
mb_used     49745.73
is_ultra        1.00
dtype: float64

In [ ]:
df.min()

calls       0.0
minutes     0.0
messages    0.0
mb_used     0.0
is_ultra    0.0
dtype: float64

In [ ]:
df['is_ultra'].mean()

0.30647168637212197

Пропусков нет, аномальных значений нет. Можно работать дальше.

## Разбивка данных на выборки

In [ ]:
df_train, df_valid = train_test_split(df, test_size = 0.2, random_state = 4)

In [ ]:
df_train, df_test = train_test_split(df_train, test_size = 0.2, random_state = 4)

In [ ]:
len(df_train) + len(df_test) + len(df_valid) == len(df)

True

In [ ]:
features_train = df_train.drop('is_ultra', axis = 1)
features_valid = df_valid.drop('is_ultra', axis = 1)
features_test = df_test.drop('is_ultra', axis = 1)
target_train = df_train['is_ultra']
target_valid = df_valid['is_ultra']
target_test = df_test['is_ultra']

Разбиваю на три выборки: 60% тест, 20% тренировочная, 20% валидационная. Проверяю, чтобы количество строк всех выборок сходилась с основным дф.

## Исследование моделей

Задача классификации, поэтому рассматриваю три модели:
 - Решающее дерево
 - Случайный лес
 - Логистическая регрессия

### Решающее дерево

In [ ]:
criterion = ['gini', 'entropy']
best_accuracy = 0.0
best_criterion = ['']
best_depth = 0
for a in criterion:
    for i in range(1,10):
        model = DecisionTreeClassifier(random_state = 4, max_depth = i, criterion = a)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_criterion = a
            best_depth = i
print(best_criterion, best_depth, best_accuracy)

gini 4 0.807153965785381


Решающее дерево имеет точность 0.807. Наилучшие параметры:
- criterion = 'gini'
- max_depth = 4

### Случайный лес

In [ ]:
criterion = ['gini', 'entropy']
best_accuracy = 0.0
best_criterion = ['']
best_depth = 0
best_est = 0
for a in criterion:
    for n in range(1, 51, 1):
        for i in range(1,10):
            model = RandomForestClassifier(random_state = 4, criterion = a, max_depth = i, n_estimators = n)
            model.fit(features_train, target_train)
            predictions = model.predict(features_valid)
            accuracy = accuracy_score(target_valid, predictions)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_criterion = a
                best_depth = i
                best_est = n
print(best_criterion, best_depth, best_est, best_accuracy)

entropy 6 19 0.8304821150855366


Случайный лес имеет точность 0.83. Наилучшие параметры:
- criterion = 'entropy
- max_depth = 6
- n_estimators = 19

### Логистическая регрессия

In [ ]:
model = LogisticRegression(random_state = 4, max_iter = 10000)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
accuracy = accuracy_score(target_valid, predictions)
accuracy

0.6967340590979783

Логистическая регрессия имеет точность 0.69.

## Проверка модели на тестовой выборке

In [ ]:
model = RandomForestClassifier(random_state = 4, criterion = 'entropy', max_depth = 6, n_estimators = 19)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
accuracy = accuracy_score(target_test, predictions)
accuracy

0.8

На тестовой выборке случайный лес получил точность 80%.

## Проверка модели на адекватность

In [ ]:
model = DummyClassifier()

In [ ]:
model.fit(features_train, target_train)

/usr/local/lib/python3.7/dist-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)


DummyClassifier(constant=None, random_state=None, strategy='warn')

In [ ]:
accuracy_score(target_test,model.predict(features_test))

0.629126213592233

## Вывод

В процессе исследования были выполнены следующие работы:

- файл с данными изучен, не обнаружено аномально низких или высоких значений, так же не обнаружено пропусков;
- данные разбиты на три выборки с соотношениями: 
    - 60% тренировочная;
    - 20% валидационная;
    - 20% тестовая;
- рассмотрены три модели со следующими результатами точности:
    - решающее дерево 80.7%;
    - случайный лес 83%;
    - логистическая регрессия 74.1%;
- для дальнейшей работы выбрана модель случайный лес со следующими параметрами: 
    - criterion = entropy;
    - max_depth = 6;
    - n_estimators = 19;
- на тестовой выборке случайный лес показал точность 80%;
- проведена проверка на адекватность (рассчитана точность модели на тестовой выборке, состоящей только из нулевых значений меток класса), получена точность 92.4%